## Detection and camera control/ person following inside Jupyter Notebook

In [1]:
import sys
from tello import Tello
import cv2
import numpy as np
import jetson.inference
import jetson.utils
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

In [2]:
# screen width and height
w,h = 240, 180  # keep in mind, that higher values of width and height will lead to freezes in image widget rendering
# in our experiment values higher than 300 showed considerably low FPS 

# PID controller parameters
#kp ,kd ,ki
#pid=[0.2,0.2,0]
pid=[0.5,0.5,0]
pError=0

pid_fb=[0.5,0.5,0]
pError_fb=0

# range of image area for forward and backward movement
area_range=[13000,25000]
dArea=13000

In [3]:
# This variable is used to prevent simultaneous command sending from 2 threads
# as it may cause drone instability and uncontrolled movements
drone_busy=False
drone_took_off=False

In [4]:
# create Tello drone instance
me =Tello(tello_ip='192.168.42.121') # the IP address of drone may change, ensure that drone in AP mode
# i e it does not create a connection Wifi (Station Mode), but connects itself to the selected Wifi
# get the IP of the drone from the wireless network provider

# set yaw_velocity default value to 0
me.yaw_velocity=0

# connect to drone
me.connect()
# print drone state and battery information
# state information contains battery information
print(me.get_battery())
print(me.get_current_state())

# start stream for getting video
me.stream_on()

2022-02-18 18:12:50,036 [INFO] tello.py - Tello instance was initialized. tello_ip: '192.168.42.121'. Port: '8889'.
2022-02-18 18:12:50,447 [INFO] tello.py - SDK mode successfully started
2022-02-18 18:12:50,852 [INFO] tello.py - SDK version: 30
2022-02-18 18:12:50,956 [INFO] tello.py - Hardware: TELLO
2022-02-18 18:12:51,062 [INFO] tello.py - Battery percentage: 100
2022-02-18 18:12:51,272 [INFO] tello.py - Video stream enabled


100
{'mid': -1, 'x': -100, 'y': -100, 'z': -100, 'mpry': '0,0,0', 'pitch': 0, 'roll': 0, 'yaw': 13, 'vgx': 0, 'vgy': 0, 'vgz': 0, 'templ': 77, 'temph': 79, 'tof': 10, 'h': 0, 'bat': 100, 'baro': 180.47, 'time': 28, 'agx': -2.0, 'agy': 0.0, 'agz': -1002.0}


In [5]:
# create network instance of SSD-mobilenet-v2 network with precision threshold equal to 0.5
net = jetson.inference.detectNet("ssd-mobilenet-v2", threshold=0.5)

In [6]:
# this function stops the drone's yaw movement
# when no person is detected
def stop_tracking(myDrone):

# set yaw_velocity to 0
    myDrone.yaw_velocity = 0

# send the set value to drone via Tello's joystick_control command
    send_command_js(0,0)#myDrone.yaw_velocity)#myDrone.joystick_control(0,0,myDrone.yaw_velocity,0)

In [7]:
# this function receives drone instance, information about center coordinates (x,y) of person, pid k values
# and the last error value, calculas yaw speed and new error, sends this yaw speed to drone (or stop movement
# signal, in case of missing detection), and returns new error value back to calling thread
def trackPerson(myDrone, center, area, w, pid, pError,pid_fb, pError_fb):
    #print("start tracking")
    # calculate new PID error and speed based on the center x value of detected Person and the center of image
    error=center[0] -w//2
    speed=pid[0]*error+pid[1]*(error-pError)
    speed=int(np.clip(speed,-100,100))

    #print(speed)
    error_fb=dArea-area
    print("Area is ", area)
    print("dArea is ", dArea)
    #fb=pid[0]*(error_fb-pError_fb)
    fb=pid_fb[0]*error_fb#   (error_fb-pError_fb)
    fb=fb/100
    fb=int(np.clip(fb,-100,100))
    
    #fb=0
    
#     if area < area_range[1] and area > area_range[0]:
#         fb=0
#     elif area > area_range[1]:
#         fb=-20
#     elif area < area_range[0]:
#         fb=20

    # if there is a detection, value of center is different from 0
    if center[0]==0:
        #myDrone.yaw_velocity= speed
    #else:
    # in case of missing detection stop moving in yaw space
        #myDrone.yaw_velocity = 0
        error=0
        error_fb=0
    print("sending yaw velocity, ", speed)#myDrone.yaw_velocity)
    print("sending pitch velocity, ", fb)#myDrone.yaw_velocity)
    fb=0
    # 30000 area is 1 meter
    # 13000 area is 2 meter
    # chack for possibility of sending joystick_control command and send the calculated speed
    #if myDrone.joystick_control:
    #send_command("joystick_control",0)
    send_command_js(speed,fb)#myDrone.yaw_velocity)#myDrone.joystick_control(0,0,myDrone.yaw_velocity,0)
    # return error for the next iteration
    return [error,error_fb]

In [8]:
#wrapper function for handling control input from widgets
def send_command(command, *args):
    global drone_busy
    global drone_took_off
    if command=="move_left":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_left(args[0])
                drone_busy=False
                break
    elif command=="move_right":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_right(args[0])
                drone_busy=False
                break
    elif command=="move_up":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_up(args[0])
                drone_busy=False
                break
    elif command=="move_down":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_down(args[0])
                drone_busy=False
                break
                
    elif command=="move_forward":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_forward(args[0])
                drone_busy=False
                break
    elif command=="move_backward":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.move_backward(args[0])
                drone_busy=False
                break
    elif command=="rotate_counterclockwise":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.rotate_counterclockwise(args[0])
                drone_busy=False
                break
    elif command=="rotate_clockwise":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.rotate_clockwise(args[0])
                drone_busy=False
                break
    elif command=="land":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.land()
                drone_took_off=False
                drone_busy=False
                break
    elif command=="takeoff":
        while True:
            if(drone_busy==False):
                drone_busy=True
                me.takeoff()
                drone_took_off=True
                drone_busy=False
                break
    #elif command=="joystick_control":
    #    while True:
    #        if(drone_busy==False):
    #            drone_busy=True
                #me.joystick_control(0,0,0,0)
    #            me.joystick_control(0,0,args[0],0)
                #me.joystick_control(0,args[1],args[0],0)
                #if(args[1])>0:
                 #   me.move_forward(args[1])
                #elif (args[1])<0:
                #     me.move_backward(abs(args[1])
    #            drone_busy=False
     #           break
    drone_busy=False
    
                

In [9]:
def send_command_js(yaw, pitch):
    global drone_busy
    while True:
        if(drone_busy==False and drone_took_off==True):
            drone_busy=True
            me.joystick_control(0,0,0,0)
            me.joystick_control(0,0,yaw,0)
            me.joystick_control(0,pitch,yaw,0)
#             if pitch>0:
#                 me.move_forward(pitch)
#             elif pitch<0:
#                 me.move_backward(abs(pitch))
#             drone_busy=False
            break
    drone_busy=False




In [10]:
# this function uses Button press information from ipywidget to send basic movement commands to Tello
def navigate(key):
    print(key)
    # using global drone value
    global me
    global drone_busy
    # setting distance (in cm) and angle (in degrees) values for movements
    distance=30
    angle=20

    # checking for code and sending the command
    if key=="left" : send_command("move_left",distance)
    elif key=="right" :send_command("move_right",distance)

    if key=="forward": send_command("move_forward",distance)
    elif key=="back": send_command("move_backward",distance)

    if key=="w": send_command("move_up",distance)
    elif key=="s": send_command("move_down",distance)

    if key=="a": 
        print("a rotate counter clockwise")
        send_command("rotate_counterclockwise",angle)
    elif key=="d": send_command("rotate_clockwise",angle)

    if key=="land": send_command("land")
    # some commands were deactivated and being tested right now
    #if kp.getKey("l"): me.go_xyz_speed(0,0,10,10)
    if key=="to": send_command("takeoff")
    #if kp.getKey("1"): me.send_command_with_return('downvision 0')
    #if kp.getKey("2"): me.send_command_with_return('downvision 1')

In [14]:
# create image widget for showing frames from camera
image_widget = ipywidgets.Image(format='jpeg')
#set the value of widget to converted value from bgr8 to jpeg format
image_widget.value = bgr8_to_jpeg(me.read_frame())

In [15]:
# function to set image widget value
def update_image(frame):
    image_widget.value = bgr8_to_jpeg(frame)

In [16]:
# this function is called on every button widget press (tied to on_click event listener)
def send_to_navigate(inp):
    #print(inp)
    # inp variable contains the clicked button description, so important for deciding, which command to send
    # inside navigate function
    navigate(inp.description.lower())

In [17]:
# create structure of buttons for control

from ipywidgets import Layout, Button, Box, VBox, ButtonStyle,GridBox


buttonW=Button(description='W',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
buttonW.on_click(send_to_navigate)
buttonS=ipywidgets.Button(description='S',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))
buttonS.on_click(send_to_navigate)
buttonA=ipywidgets.Button(description='A',
                 layout=Layout(width='auto', grid_area='main'),
                 style=ButtonStyle(button_color='moccasin'))
buttonA.on_click(send_to_navigate)
buttonD=ipywidgets.Button(description='D',
                 layout=Layout(width='auto', grid_area='sidebar'),
                 style=ButtonStyle(button_color='salmon'))
buttonD.on_click(send_to_navigate)


buttonU=ipywidgets.Button(description='FORWARD',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
buttonU.on_click(send_to_navigate)
buttonDown=ipywidgets.Button(description='BACK',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))
buttonDown.on_click(send_to_navigate)
buttonL=ipywidgets.Button(description='LEFT',
                 layout=Layout(width='auto', grid_area='main'),
                 style=ButtonStyle(button_color='moccasin'))
buttonL.on_click(send_to_navigate)
buttonR=ipywidgets.Button(description='RIGHT',
                 layout=Layout(width='auto', grid_area='sidebar'),
                 style=ButtonStyle(button_color='salmon'))
buttonR.on_click(send_to_navigate)


buttonland=ipywidgets.Button(description='LAND',layout=Layout(width='100px'),
                 style=ButtonStyle(button_color='red'))
buttonland.on_click(send_to_navigate)
buttontakeoff=ipywidgets.Button(description='TO',layout=Layout(width='100px'),
                 style=ButtonStyle(button_color='blue'))
buttontakeoff.on_click(send_to_navigate)


gr1=GridBox(children=[buttonW, buttonA, buttonD, buttonS],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            ". header ."
            "main . sidebar "
            ". footer ."
            ''')
       )
gr2=GridBox(children=[buttonU, buttonDown, buttonL, buttonR],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            ". header ."
            "main . sidebar "
            ". footer ."
            ''')
       )
all_widget = ipywidgets.HBox([
  gr1,ipywidgets.VBox([buttonland,buttontakeoff]),  gr2  

])

In [18]:
# Python's threading module is utilised  to run detection in own thread
import threading

In [19]:
# the detection part was converted into function for threading purpose
def detect_camera():
    global pError, pError_fb
    global drone_busy
    while True:
        # read frame from Tello
        frame=me.read_frame()


        if (frame is not None):
            # resize frame image to width and height, given above
            img = cv2.resize(frame, (w, h))
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # change the image format from Numpy array to CUDA (used for jetson detection)
            cuda_image=jetson.utils.cudaFromNumpy(img)
            # detect objects from CUDA image
            detections = net.Detect(cuda_image)

            # flag for checking if object was found
            object_found = False
            # list for center coordinates of object
            objectsListC = []
            # list for area values of detections
            objectsArea = []
            # iterate over detections
            for detection in detections:
                # get class name from ID
                cl_name=net.GetClassDesc(detection.ClassID)
                # get detection center coordinates
                center=detection.Center
                # check for class name, only person is required in this implementation
                if (cl_name=="person"):
                    # set the flag to true
                    object_found=True
                    # print detection values
                    #print("Top: {}, Bottom: {}, Left: {}, Right: {}, Height: {}, Width: {}, Area: {}, Center: {}, ".format(detection.Top,detection.Bottom,detection.Left,detection.Right,detection.Height,detection.Width, detection.Area,detection.Center))
                    # append found detection center and area to responding lists
                    objectsListC.append(detection.Center)
                    objectsArea.append(detection.Area)
            # if there were detections found
            #stop_tracking(me)
            if len(objectsArea) != 0:
                # get the center value with maximum area
                i = objectsArea.index(max(objectsArea))
                # send tracking command and update PID error value
                #print("Area is ",objectsArea[0])
                pError, pError_fb = trackPerson(me, objectsListC[i],objectsArea[i], w, pid, pError, pid_fb, pError_fb)
            # if object was not found
            #if object_found == False:
            #stop drone movement
                #stop_tracking(me)

            # test code for understanding detection objects inside structure
            #print(dir(detection))
            #['Area', 'Bottom', 'Center', 'ClassID', 'Confidence', 'Contains', 'Height', 'Instance', 'Left', 'ROI', 'Right', 'Top', 'Width', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

            #Top: 214.1015625, Bottom: 655.6640625, Left: 672.1875, Right: 841.875, Height: 441.5625, Width: 169.6875, Area: 74927.640625, Center: (757.03125, 434.8828125), 

            # convert the cuda image with detections on it back to numpy image
            nmp_img=jetson.utils.cudaToNumpy(cuda_image)
            # show that numpy image on desktop
            if len(objectsArea) != 0:
                cv2.putText(nmp_img, 
                     str(int(objectsArea[0])), 
                     (10, 19), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                     (0, 255, 255), 
                     2, 
                     cv2.LINE_4)
            update_image(nmp_img)

In [20]:
# creates new thread and passes the detection function 'detect_camera' as an argument
new_thread = threading.Thread(target=detect_camera)
# starts detection thread as an input
new_thread.start()

In [21]:
# displays frames inside python cell
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [22]:
# displays control widgets inside python cell
display(all_widget)


Area is  1486.1058349609375
dArea is  13000
sending yaw velocity,  100
sending pitch velocity,  57
